<a href="https://colab.research.google.com/github/joshibrom/Machine-Learning-Tweet-Classification/blob/main/Machine_Learning_Project_Final_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Group Members: Abigail Amiscosa, Joshua Ibrom, Julian Spindola

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "tweets.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "vstepanenko/disaster-tweets",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and testing (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Print some basic info to confirm
print("Training set size:", len(train_df))
print("Testing set size:", len(test_df))

# print("\nSample training data:")
# print(train_df.head())

# print("\nSample testing data:")
# print(test_df.head())


In [ ]:
import re
import string
from nltk.corpus import stopwords
import nltk

# Download NLTK stopwords (only once)
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)  # remove URLs
    text = re.sub(r"@\w+", "", text)                     # remove mentions
    text = re.sub(r"#\w+", "", text)                     # remove hashtags
    text = text.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    text = text.lower()                                  # lowercase
    text = " ".join([word for word in text.split() if word not in stop_words])  # remove stopwords
    return text.strip()

# Create the new 'clean_text' column
df['clean_text'] = df['text'].apply(clean_text)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud

# Make sure your dataset is loaded
# Example: df = train_df or the original df

# Basic dataset overview
print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nMissing values per column:\n", df.isnull().sum())
print("\nClass distribution:\n", df['target'].value_counts())

# Plot class distribution (0 = Not Disaster, 1 = Disaster)
plt.figure(figsize=(6,4))
sns.countplot(x='target', data=df)
plt.title("Distribution of Disaster vs. Non-Disaster Tweets")
plt.xlabel("Target (0 = Non-Disaster, 1 = Disaster)")
plt.ylabel("Count")
plt.show()

# Tweet length distribution
df['text_length'] = df['text'].apply(len)

plt.figure(figsize=(8,4))
sns.histplot(df['text_length'], bins=30, kde=True)
plt.title("Distribution of Tweet Lengths")
plt.xlabel("Tweet Length (characters)")
plt.ylabel("Frequency")
plt.show()

# Most common words
disaster_words = " ".join(df[df['target'] == 1]['clean_text'])
nondisaster_words = " ".join(df[df['target'] == 0]['clean_text'])

# Generate word clouds
wordcloud_disaster = WordCloud(width=800, height=400, background_color='white').generate(disaster_words)
wordcloud_non = WordCloud(width=800, height=400, background_color='white').generate(nondisaster_words)

# Show them side-by-side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(wordcloud_disaster, interpolation='bilinear')
axes[0].set_title("Word Cloud – Disaster Tweets")
axes[0].axis('off')

axes[1].imshow(wordcloud_non, interpolation='bilinear')
axes[1].set_title("Word Cloud – Non-Disaster Tweets")
axes[1].axis('off')

plt.show()


In [ ]:
# 1. Data Loading and Exploration

# Install dependencies if needed:
# pip install kagglehub[pandas-datasets] torch torchvision transformers scikit-learn bert-score matplotlib
!pip install bert-score

import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import string
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from torch.optim import AdamW
from bert_score import score as bert_score

# Load dataset
file_path = "tweets.csv"

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "vstepanenko/disaster-tweets",
    file_path,
)

# print("First 5 records:")
# print(df.head())
# print("\nDataset info:")
# print(df.info())

In [ ]:
# 2. Text Preprocessing

def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # remove URLs
    text = re.sub(r"@\w+", "", text)     # remove mentions
    text = re.sub(r"#\w+", "", text)     # remove hashtags
    text = text.translate(str.maketrans("", "", string.punctuation))  # remove punctuation
    text = text.lower().strip()
    return text

df["text"] = df["text"].astype(str).apply(clean_text)

In [ ]:
# 3. Feature Engineering & Vectorization

# Split data
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["target"], test_size=0.2, random_state=42)

# CNN BASELINE (TF-IDF)
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train.tolist()).toarray()
X_test_tfidf = vectorizer.transform(X_test.tolist()).toarray()

X_train_tensor = torch.tensor(X_train_tfidf, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_tfidf, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32)

train_dataset_cnn = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset_cnn = TensorDataset(X_test_tensor, y_test_tensor)

In [ ]:
# 4. Training - CNN Baseline

class CNNBaseline(nn.Module):
    def __init__(self, input_dim, num_classes=2):
        super(CNNBaseline, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=5)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.fc = nn.Linear(16 * ((input_dim - 5 + 1)//2), num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        return self.fc(x)

cnn_model = CNNBaseline(input_dim=X_train_tfidf.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(cnn_model.parameters(), lr=1e-3)

epochs = 3
cnn_model.train()
for epoch in range(epochs):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = cnn_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[CNN] Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

# Evaluate CNN
cnn_model.eval()
y_pred_cnn = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        outputs = cnn_model(X_batch)
        y_pred_cnn.extend(torch.argmax(outputs, dim=1).cpu().numpy())

print("\n=== CNN Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred_cnn))
print("Precision:", precision_score(y_test, y_pred_cnn))
print("Recall:", recall_score(y_test, y_pred_cnn))
print("F1:", f1_score(y_test, y_pred_cnn))


In [ ]:
# 5. Training - BERT Model

# Install bert-score library
!pip install bert-score

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set checkpoint directory
import os
checkpoint_dir = "/content/drive/MyDrive/Disaster_BERT_Checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Tokenizer
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from bert_score import score as bert_score

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch, padding=True, truncation=True, max_length=64, return_tensors="pt")

train_encodings = tokenize(X_train.tolist())
test_encodings = tokenize(X_test.tolist())

train_dataset = TensorDataset(
    train_encodings["input_ids"],
    train_encodings["attention_mask"],
    torch.tensor(y_train.values)
)
test_dataset = TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
    torch.tensor(y_test.values)
)

train_loader_bert = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader_bert = DataLoader(test_dataset, batch_size=8)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)
optimizer = AdamW(bert_model.parameters(), lr=2e-5)

# Training
epochs = 2
best_f1 = 0.0

bert_model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader_bert:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader_bert)
    print(f"[BERT] Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    # Evaluate after epoch
    bert_model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in test_loader_bert:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            outputs = bert_model(input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            trues.extend(labels.cpu().numpy())

    epoch_f1 = f1_score(trues, preds)
    print(f"Epoch {epoch+1} F1 Score: {epoch_f1:.4f}")

    # Save checkpoint to Drive
    save_path = os.path.join(checkpoint_dir, f"bert_checkpoint_epoch{epoch+1}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': bert_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': avg_loss,
    }, save_path)
    print(f"✅ Saved checkpoint to Google Drive: {save_path}")

    # Save best model
    if epoch_f1 > best_f1:
        best_f1 = epoch_f1
        best_path = os.path.join(checkpoint_dir, "bert_best_model.pt")
        torch.save(bert_model.state_dict(), best_path)
        print(f"🌟 New best model saved at epoch {epoch+1} (F1={best_f1:.4f})")

    bert_model.train()

# Final evaluation
bert_model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch in test_loader_bert:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        outputs = bert_model(input_ids, attention_mask=attention_mask)
        preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
        trues.extend(labels.cpu().numpy())

print("\n=== BERT Evaluation ===")
print("Accuracy:", accuracy_score(trues, preds))
print("Precision:", precision_score(trues, preds))
print("Recall:", recall_score(trues, preds))
print("F1:", f1_score(trues, preds))

# BERTScore (semantic similarity check)
P, R, F1_score_val = bert_score(X_test.tolist()[:50], X_test.tolist()[:50], lang="en", verbose=False)
print("\nBERTScore (self-similarity check):", F1_score_val.mean().item())

# Save final model and tokenizer to Drive
final_dir = os.path.join(checkpoint_dir, "bert_disaster_final")
bert_model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
print(f"✅ Final fine-tuned BERT model and tokenizer saved to Google Drive at: {final_dir}")


In [ ]:
# 6. BERTScore Evaluation

# Extract tweets the model predicted as disaster (label = 1)
predicted_disaster_texts = X_test[np.array(preds) == 1]

# Extract tweets that are truly disaster (label = 1)
true_disaster_texts = X_test[y_test.values == 1]

sample_size = min(50, len(predicted_disaster_texts), len(true_disaster_texts))

predicted_samples = predicted_disaster_texts[:sample_size].tolist()
true_samples = true_disaster_texts[:sample_size].tolist()

print(f"\nComputing BERTScore on {sample_size} predicted-vs-true disaster tweets...")

P, R, F1 = bert_score(predicted_samples, true_samples, lang="en", verbose=True)

print("\n=== Semantic BERTScore Evaluation ===")
print(f"Precision: {P.mean():.4f}")
print(f"Recall:    {R.mean():.4f}")
print(f"F1 Score:  {F1.mean():.4f}")

print("\nInterpretation:")
print("• Higher scores mean predicted disaster tweets are semantically similar to real disaster tweets.")
print("• This helps evaluate whether the model is capturing disaster-related meaning, not just labels.")


In [ ]:
# 7. Hyperparameter Tuning & Architecture Experiments

# 7.1 CNN Hyperparameter Tuning (Batch Size & LR)

cnn_experiment_results = []

batch_sizes = [16, 32]
learning_rates = [0.001, 0.0005]

print("Running CNN Hyperparameter Experiments...\n")

for bs in batch_sizes:
    for lr in learning_rates:
        print(f"Testing CNN with batch_size={bs}, learning_rate={lr}")

        # Create DataLoaders for this experiment
        train_loader_exp = DataLoader(train_dataset_cnn, batch_size=bs, shuffle=True)
        test_loader_exp = DataLoader(test_dataset_cnn, batch_size=bs)

        model_exp = CNNBaseline(input_dim=X_train_tfidf.shape[1]).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer_exp = torch.optim.Adam(model_exp.parameters(), lr=lr)

        # Train
        model_exp.train()
        for batch_x, batch_y in train_loader_exp:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer_exp.zero_grad()
            outputs = model_exp(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer_exp.step()

        # Evaluate
        model_exp.eval()
        preds_exp = []
        trues_exp = []
        with torch.no_grad():
            for batch_x, batch_y in test_loader_exp:
                batch_x = batch_x.to(device)
                outputs = model_exp(batch_x)
                preds_exp.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                trues_exp.extend(batch_y.numpy())

        acc = accuracy_score(trues_exp, preds_exp)
        f1 = f1_score(trues_exp, preds_exp)

        cnn_experiment_results.append({
            "batch_size": bs,
            "learning_rate": lr,
            "accuracy": acc,
            "f1": f1
        })

        print(f" -> Accuracy: {acc:.4f}, F1: {f1:.4f}\n")

# 7.2 CNN Architecture Variation (Extra Conv Layer)

print("\nTesting CNN Architecture Variation: Extra Convolution Layer...\n")

class CNNVariant(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

cnn_variant = CNNVariant(X_train_tfidf.shape[1]).to(device)
optimizer_var = torch.optim.Adam(cnn_variant.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# train
cnn_variant.train()
for batch_x, batch_y in train_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    optimizer_var.zero_grad()
    outputs = cnn_variant(batch_x)
    loss = criterion(outputs, batch_y)
    loss.backward()
    optimizer_var.step()

# evaluate
cnn_variant.eval()
v_preds, v_true = [], []
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        outputs = cnn_variant(batch_x)
        v_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        v_true.extend(batch_y.numpy())

variant_accuracy = accuracy_score(v_true, v_preds)
variant_f1 = f1_score(v_true, v_preds)

print(f"Variant CNN Accuracy: {variant_accuracy:.4f}")
print(f"Variant CNN F1:       {variant_f1:.4f}")

# 7.3 BERT Hyperparameter Tuning (Learning Rate)

print("\nRunning BERT Hyperparameter Experiments...\n")

bert_learning_rates = [2e-5, 3e-5]

bert_results = []

for lr in bert_learning_rates:
    print(f"Testing BERT with learning_rate={lr}")

    bert_small = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=2
    ).to(device)

    optimizer_small = torch.optim.AdamW(bert_small.parameters(), lr=lr)

    # train
    bert_small.train()
    for batch in train_loader_bert:
        ids, mask, labels = [x.to(device) for x in batch]
        optimizer_small.zero_grad()
        output = bert_small(ids, attention_mask=mask, labels=labels)
        loss = output.loss
        loss.backward()
        optimizer_small.step()

    # evaluate
    bert_small.eval()
    preds_b, trues_b = [], []
    with torch.no_grad():
        for batch in test_loader_bert:
            ids, mask, labels = [x.to(device) for x in batch]
            output = bert_small(ids, attention_mask=mask)
            preds_b.extend(torch.argmax(output.logits, dim=1).cpu().numpy())
            trues_b.extend(labels.cpu().numpy())

    acc = accuracy_score(trues_b, preds_b)
    f1 = f1_score(trues_b, preds_b)

    bert_results.append({
        "learning_rate": lr,
        "accuracy": acc,
        "f1": f1
    })

    print(f" -> Accuracy: {acc:.4f}, F1: {f1:.4f}\n")

# 7.4 Results Summary Tables

print("\n===== CNN Hyperparameter Results =====")
cnn_results_df = pd.DataFrame(cnn_experiment_results)
print(cnn_results_df)

print("\n===== CNN Architecture Variant =====")
print(f"Accuracy: {variant_accuracy:.4f}, F1: {variant_f1:.4f}")

print("\n===== BERT Hyperparameter Results =====")
bert_results_df = pd.DataFrame(bert_results)
print(bert_results_df)


In [ ]:
# 8. Demo of Classifying Tweets
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F

# LOAD DATASET
# pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "tweets.csv"

print("\nLoading dataset...")
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "vstepanenko/disaster-tweets",
    file_path,
)

# Convert df.head() into list of Python dictionaries
head_list = df.head().to_dict(orient="records")

# BERT PREDICTION FUNCTION
best_model_path = "/content/drive/MyDrive/Disaster_BERT_Checkpoints/bert_best_model.pt"

def demo_bert_best(tweet_text):
    print("\nLoading BEST model weights...")

    # 1. Build base model architecture
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased",
        num_labels=2
    )
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    # 2. Load trained state_dict
    model.load_state_dict(torch.load(best_model_path, map_location=torch.device('cpu')))
    model.eval()

    # 3. Tokenize input
    inputs = tokenizer(
        tweet_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=64
    )

    with torch.no_grad():
        outputs = model(**inputs)

    # 4. Softmax → label + confidence
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    pred_label = torch.argmax(probs, dim=1).item()
    confidence = probs[0][pred_label].item()

    label_map = {1: "🚨 DISASTER", 0: "❌ NON-DISASTER"}

    print(f"\n[BEST MODEL] Tweet: {tweet_text}")
    print(f"Result: {label_map[pred_label]} ({confidence:.2%})")


# RUN MODEL ON FIRST 5 TWEETS FROM DATASET

print("\n===== RUNNING MODEL ON FIRST 5 TWEETS =====")
for i, row in enumerate(head_list):
    print(f"\n Tweet {i+1} ")
    demo_bert_best(row["text"])
